# UDF Testing in the Notebook

This notebook shows how to write and test BigQuery Javascript UDFs (user-defined functions) within a notebook.

Before using this tutorial you should go through the [UDFs in BigQuery](notebooks/datalab/tutorials/BigQuery/UDFs%20in%20BigQuery.ipynb) tutorial which discusses how to use UDFs in notebooks without external code.

You can read more about UDFs [here](https://cloud.google.com/bigquery/user-defined-functions). 



## Scenario

This notebook uses the same scenario as before, looking at some anonymized logs that originated in Google AppEngine. 

## Creating and Testing the UDF

UDFs are Javascript functions that take a row object and emitter function as input; they perform some computation and then call the emitter function to output a result row object. We will first write and test our UDF as Javascript. We can use a %%javascript cell for this. We will write the code of the UDF (and its helper function `getParameters`), then define some test data, and finally call the UDF with the test data after creating a mock version of the emitter function. Note that in a %%javascript cell you can access the output area element with the name `element`; our mock emitter will use that to produce output we can see in the notebook.

There is no state shared between %%javascript cells so we have to do this all in a single cell.

In [1]:
%%javascript

/**
 * A helper function to split a set of URL query parameters into an object
 * as key/value properties.
 */
function getParameters(path) {
  var re = /[?&]([^=]*)=([^&]*)/g;
  var result = {};
  var match;
  while ((match = re.exec(path)) != null) {
    result[match[1]] = decodeURIComponent(match[2]);
  }
  return result;  
}

/**
 * Our UDF function, which takes a row r and emitter function emitFn.
 * We assume each row r has the five columns from our input (timestamp, 
 * method, latency, status and path). We will parse path from the input
 * and add its constituent parts, then call the emitter.
 *
 * Note: we re-use r for the output as we are keeping its fields but we
 * could have created a new object if that was more appropriate.
 */
var udf = function(r, emitFn) {
  var q = getParameters(r.path);
  var split = r.path.indexOf('?');
  r.event = r.path.substr(5, split - 5);
  r.project = q.project;
  r.instance = q.instance;
  r.user = q.user;
  r.page = q.page;
  r.path = q.path;
  r.version = q.version;
  r.release = q.release;
  emitFn(r);
}

// Now we want to test the UDF. We can try calling it using a sample line from our table.
// Note that the variable 'element' is available to us to create output in the notebook,
// so our test emitter will use that to display the fields.
// Also note that the Date function in Javascript uses 0-based months so October is 9 (wat!)

var test_row = {
  latency: 0.00311,
  method: 'POST',
  path: '/log/page?project=14&instance=81&user=16&page=master&path=63&version=0.1.1&release=alpha',
  status: 204,
  timestamp: new Date(2015, 9, 27, 2, 41, 20, 430256)
};

function emitter(r) {
  for (var p in r) {
    element.append(p + '=' + r[p] + '<br>');
  }
}

udf(test_row, emitter);


<IPython.core.display.Javascript object>

Looks like we are good to go!


## Next Steps

If you have code that you use regularly in your UDFs you can factor that out and put it in Google Cloud Storage then import it; this is covered in the tutorial  [UDFs using Code in Cloud Storage](notebooks/datalab/tutorials/BigQuery/UDFs%20using%20Code%20in%20Cloud%20Storage.ipynb).